In [4]:
:dep plotly
:dep serde_json
:dep serde
:dep parquet

In [5]:

use std::collections::HashMap;
use serde::{Deserialize, Serialize};
use serde_json::{Value, json};


use parquet::file::properties::WriterProperties;
use parquet::file::writer::{FileWriter, SerializedFileWriter};
use parquet::schema::parser::parse_message_type;

use std::fs;
use std::path::Path;

In [6]:
json!({"test": 1})

Object({"test": Number(1)})

In [8]:

#[derive(Debug)]
struct Schema {
    pub schema: String,
    pub cols: Vec<String>
}

impl Schema {
    pub fn from_value(value: Value) -> Self {
        let mut schema: String = String::from("message schema {");
        let mut cols: Vec<String> = vec![];

        if value.is_object() {
            match value {
                Value::Object(map) => {
                    for (k, v) in map {
                        let schema_key: String = String::from(k.to_owned());
                        cols.push(k.clone());
                        let mut finalstr = format!("\n \trequired");
                        match v {
                            Value::Number(number) => {
                                let mut num_type: String = String::default();
                                    if number.is_i64() {
                                      num_type = String::from("INT64");
                                    } else if number.is_u64() {
                                       num_type = String::from("INT64");
                                      
                                    } else if number.is_f64() {
                                       num_type = String::from("DOUBLE");
                                    } else {
                                        num_type = String::from("DOUBLE");
                                    }
                                
                                finalstr = format!("{} {} {};", finalstr, num_type,  schema_key);
                 
                            },
                            Value::String(_v_str) => {
                                finalstr = format!("{} BINARY {} (UTF8);", finalstr, schema_key);
                                
                            },
                            Value::Bool(_boolean) => { finalstr = format!("{} BOOLEAN {};", finalstr, schema_key) },
                            _ => ()

                        };
                        
                    
                        
             
                    
                    schema.push_str(&finalstr.to_owned());
                    }
                },
                _=>{}
            }
        } 
        
        schema.push_str(&"\n}".to_owned());

        Schema{
            schema,
            cols
        }
    }
}


In [14]:
fn write_to_file(loc: &str, data: HashMap<String, Vec<Value>>) {

    let path = Path::new(loc);

    // Generate Schema
    let message_type = Schema::from_value(json!({"a": 1}));

    let schema = std::sync::Arc::new(parse_message_type(message_type.schema.as_str()).unwrap());
    let props = std::sync::Arc::new(WriterProperties::builder().build());
    let file = fs::File::create(&path).unwrap();
    let mut writer = SerializedFileWriter::new(file, schema, props).unwrap();
    let mut row_group_writer = writer.next_row_group().unwrap();

    let mut row_id = 0;

    while let Some(mut col_writer) = row_group_writer.next_column().unwrap() {
        // ... write values to a column writer
        match col_writer {
            parquet::column::writer::ColumnWriter::Int32ColumnWriter(ref mut typed_writer) => {
                typed_writer.write_batch(&[1], None, None).unwrap();
            },
            parquet::column::writer::ColumnWriter::Int64ColumnWriter(ref mut _tw) => {

            },
            parquet::column::writer::ColumnWriter::Int96ColumnWriter(ref mut _tw) => {

            },
            parquet::column::writer::ColumnWriter::FloatColumnWriter(ref mut _tw) => {

            },
            parquet::column::writer::ColumnWriter::DoubleColumnWriter(ref mut _tw) => {

            },
            parquet::column::writer::ColumnWriter::ByteArrayColumnWriter(ref mut _typed_writer) => {
                println!("BYTE ARRAY");
                let buf: Vec<u8> = String::from("hi").as_bytes().to_vec();
                _typed_writer.write_batch(&[parquet::data_type::ByteArray::from(buf)], None, None).unwrap();
            },
            parquet::column::writer::ColumnWriter::FixedLenByteArrayColumnWriter(ref mut _typed_writer) => {
                println!("FIXED BYTE ARRAY");
            },
            parquet::column::writer::ColumnWriter::BoolColumnWriter(ref mut _tw ) => {},
            
            
        }
        println!("COL IS {}", message_type.cols[row_id]);

        row_id = row_id + 1;
        // col_writer.
        row_group_writer.close_column(col_writer).unwrap();
    }
}

In [13]:

fn write(samples: Vec<Value>) {
    
    for sample in samples {
        
    }
}